In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import sum as _sum, avg, expr, window, from_unixtime, col

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import time, datetime
import os
import random
import uuid

spark = (
        SparkSession.builder.appName("spark application")
        .config(
            "spark.jars.packages",
            "org.apache.hudi:hudi-spark3.1-bundle_2.12:0.13.0",
        )
        .config(
            "spark.sql.extensions",
            "org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        )
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config(
            "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
        )
        # .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
        # .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
        # .config("spark.hadoop.fs.s3a.endpoint", "http://10.159.37.34:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config(
            "spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
        )
        .config("spark.hadoop.fs.s3a.connection.maximum", "1000")
        .getOrCreate()
    )

23/06/19 17:18:07 WARN Utils: Your hostname, ducdn resolves to a loopback address: 127.0.1.1; using 10.1.124.58 instead (on interface enp1s0)
23/06/19 17:18:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
org.apache.hudi#hudi-spark3.1-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eff842d3-18d6-4c1d-8a9e-7bac9613536c;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 in central
:: resolution report :: resolve 199ms :: artifacts dl 8ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-

In [4]:
df = spark.read.format("hudi").load("/home/ducdn/Desktop/workspace/hudi_mor/hudi_mor")
df.show()

# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|                  id|     value| timestamp|partition|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|  20230619152433110|20230619152433110...|99d98e7b-d5c2-431...|                     2|b2cc23ea-1b60-44c...|99d98e7b-d5c2-431...|  11223344|1687163073|        2|
|  20230619152117652|20230619152117652...|96234511-4154-48e...|                     2|b2cc23ea-1b60-44c...|96234511-4154-48e...| 123456789|1687162877|        2|
|  20230619151758093|20230619151758093...|c017eb4f-a2e5-4bc...|                     2|b2cc23ea-1b60-44c...|c017eb4f-a2e5-4bc...|1687162678| 123456789|        2|
|  20230619151342517|2023061915134

In [1]:
spark.sql("show tables").show(100, False)

NameError: name 'spark' is not defined

In [4]:
hudiIncQueryDF = spark.read \
    .format("org.apache.hudi") \
    .option("hoodie.datasource.query.type", "incremental") \
    .option("hoodie.datasource.read.begin.instanttime", "20230619172801944") \
    .load("s3a://datalake/hudi_mor")

hudiIncQueryDF.createOrReplaceTempView("hudi_mor")
spark.sql("select * from hudi_mor order by _hoodie_commit_time").show()

Py4JJavaError: An error occurred while calling o70.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2595)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3269)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.hudi.common.fs.FSUtils.getFs(FSUtils.java:110)
	at org.apache.hudi.common.fs.FSUtils.getFs(FSUtils.java:103)
	at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala:88)
	at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala